In [1]:
import os
from facenet_pytorch import MTCNN
import cv2
from tqdm.notebook import tqdm
import torch
import numpy as np

In [2]:
# Paths
dataset_root = 'CelebrityFacesDataset_Curated'
output_root = 'yolo_dataset'

train_image_output_dir = os.path.join(output_root, 'train', 'images')
train_label_output_dir = os.path.join(output_root, 'train', 'labels')
train_boxes_output_dir = os.path.join(output_root, 'train', 'box_visualizations')

val_image_output_dir = os.path.join(output_root, 'val', 'images')
val_label_output_dir = os.path.join(output_root, 'val', 'labels')
val_boxes_output_dir = os.path.join(output_root, 'val', 'box_visualizations')

os.makedirs(train_image_output_dir, exist_ok=True)
os.makedirs(train_label_output_dir, exist_ok=True)
os.makedirs(train_boxes_output_dir, exist_ok=True)

os.makedirs(val_image_output_dir, exist_ok=True)
os.makedirs(val_label_output_dir, exist_ok=True)
os.makedirs(val_boxes_output_dir, exist_ok=True)


In [3]:

# Init face detector
mtcnn = MTCNN(keep_all=False, device='cuda' if torch.cuda.is_available() else 'cpu')


c:\Users\Pruthweesha\AppData\Local\Programs\Python\Python312\Lib\site-packages\facenet_pytorch\models\mtcnn.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict =

In [4]:

no_face_virtual_label = '__NoFace'

def resize_with_padding(img, target_size=640, pad_color=(0, 0, 0)):
    h, w = img.shape[:2]
    scale = (target_size * 1.00) / max(h, w)
    new_w, new_h = int(w * scale), int(h * scale)

    # Resize the image with preserved aspect ratio
    resized = cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_LINEAR)

    # Create a new square image and paste resized image into center
    padded = np.full((target_size, target_size, 3), pad_color, dtype=np.uint8)
    x_offset = (target_size - new_w) // 2
    y_offset = (target_size - new_h) // 2
    padded[y_offset:y_offset + new_h, x_offset:x_offset + new_w] = resized

    return padded

# Build class index
celebrity_names = sorted([d for d in os.listdir(dataset_root) if os.path.isdir(os.path.join(dataset_root, d)) and not d.startswith('_')])
celebrity_names = sorted([d for d in os.listdir(dataset_root) if os.path.isdir(os.path.join(dataset_root, d)) and d.startswith('_')], reverse=True) + celebrity_names
celebrity_names.remove(no_face_virtual_label)

classes_ignored = {"DenzelWashington", "HughJackman", "TomCruise"}

celebrity_names = [x for x in celebrity_names if x not in classes_ignored]

class_map = {name: idx for idx, name in enumerate(celebrity_names)}

# Process images
for celeb_name in tqdm(celebrity_names):
    class_id = class_map[celeb_name]
    celeb_dir = os.path.join(dataset_root, celeb_name)
    class_corpus_size = len(os.listdir(celeb_dir))


    for image_index, img_name in enumerate(os.listdir(celeb_dir)):
        val_datapoint = image_index % 6 == 1
        image_output_dir = val_image_output_dir if val_datapoint else train_image_output_dir
        label_output_dir = val_label_output_dir if val_datapoint else train_label_output_dir
        boxes_output_dir = val_boxes_output_dir if val_datapoint else train_boxes_output_dir

        img_path = os.path.join(celeb_dir, img_name)
        if not img_path.lower().endswith(('.jpg', '.jpeg', '.png')):
            continue

        img = cv2.imread(img_path)
        if img is None:
            continue

        img = resize_with_padding(img)

        rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        boxes, _ = mtcnn.detect(rgb)

        if boxes is None:
            continue

        boxes = sorted(boxes, key=(lambda x: x[2]-x[0]), reverse=True )

        for i, box in enumerate(boxes):
            if box is None or len(box) != 4:
                continue

            x1, y1, x2, y2 = box
            h, w = img.shape[:2]

            # Clamp to image bounds
            x1 = max(0, min(x1, w - 1))
            x2 = max(0, min(x2, w - 1))
            y1 = max(0, min(y1, h - 1))
            y2 = max(0, min(y2, h - 1))

            # Reject invalid boxes
            if x2 <= x1 or y2 <= y1:
                print(f"Skipped corrupt box: {box}")
                continue

            # Normalize for YOLO
            x_center = ((x1 + x2) / 2) / w
            y_center = ((y1 + y2) / 2) / h
            width = (x2 - x1) / w
            height = (y2 - y1) / h

            # Reject extremely small boxes
            if width <= 0 or height <= 0 or width > 1 or height > 1:
                print(f"Skipped corrupt normalized box: {width=:.4f}, {height=:.4f}")
                continue

            # Save image to common output
            output_img_path = os.path.join(image_output_dir, f'{celeb_name}_{img_name}')
            cv2.imwrite(output_img_path, img)

            # Save label
            label_path = os.path.join(label_output_dir, f'{celeb_name}_{img_name}'.replace('.jpg', '.txt').replace('.png', '.txt').replace('.JPG', '.txt'))
            with open(label_path, 'w') as f:
                f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

            # Draw bounding box on a copy and save
            vis_img = img.copy()
            cv2.rectangle(vis_img, (int(x1), int(y1)), (int(x2), int(y2)), color=(0, 255, 0), thickness=2)
            vis_path = os.path.join(boxes_output_dir, f'{celeb_name}_{img_name}')
            cv2.imwrite(vis_path, vis_img)

            break


## Add no face directory
celeb_name = f"{no_face_virtual_label}"
celeb_dir = os.path.join(dataset_root, celeb_name)
for image_index,img_name in enumerate(os.listdir(celeb_dir)):
    image_output_dir = val_image_output_dir if image_index % 6 == 1 else train_image_output_dir
    label_output_dir = val_label_output_dir if image_index % 6 == 1 else train_label_output_dir
    boxes_output_dir = val_boxes_output_dir if image_index % 6 == 1 else train_boxes_output_dir

    img_path = os.path.join(celeb_dir, img_name)
    img = cv2.imread(img_path)
    if img is None:
        continue

    img = resize_with_padding(img)

    rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Save image to common output
    output_img_path = os.path.join(image_output_dir, f'{celeb_name}_{img_name}')
    cv2.imwrite(output_img_path, img)

    # Save label
    label_path = os.path.join(label_output_dir, f'{celeb_name}_{img_name}'.replace('.jpg', '.txt').replace('.png', '.txt'))
    with open(label_path, 'w') as f:
        f.write(f"\n")

  0%|          | 0/6 [00:00<?, ?it/s]